In [ ]:
# jupyter_vim

In [ ]:
# Run Script below
! python /home/app/kafka/src/create_topic.py --topic=book --file=/home/app/data/brothers-karamazov.txt --format=json

In [ ]:

! export PYTHONPATH=../risingwave:$PYTHONPATH && python ../risingwave/src/main.py --topic=book --bootstrap-servers=kafka-broker:9092 --schema='{"line": "INTEGER","text": "VARCHAR"}' --source='kafka'


In [19]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="risingwave", port=4566, user="root", dbname="dev")
conn.autocommit = True

In [20]:
with conn.cursor() as cur:
    cur.execute("SELECT kafka_timestamp, line, text FROM book LIMIT 10;")
    values = cur.fetchall()

In [ ]:
pd.DataFrame(values, columns=["timestamp", "line", "text"])

In [ ]:
# create materialized view
with conn.cursor() as cur:
    cur.execute("DROP MATERIALIZED VIEW IF EXISTS book_view")
    cur.execute(
        """
        CREATE MATERIALIZED VIEW book_view AS
        SELECT
            date_trunc('second', kafka_timestamp) AS window_start,
            SUM(LENGTH(text)) AS total_characters,
            count(*) AS total_lines
        FROM
            book
        GROUP BY
            window_start;
        """
    )

In [ ]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM book_view;")
    values = cur.fetchall()
pd.DataFrame(values, columns=["timestamp", "total_characters", "total_lines"])

In [ ]:
import duckdb

con = duckdb.connect()
con.sql(
    f"""
    INSTALL postgres;
    LOAD postgres;  
    ATTACH 'host=risingwave port=4566 dbname=dev user=root' AS db (TYPE postgres);  
"""
)
con.sql("SHOW DATABASES;")
con.sql("SHOW ALL TABLES;")